In [2]:
import pandas as pd
import requests
import folium
import geopandas as gpd
import shapely
from shapely.geometry import Polygon

### Creating DataFrame for Pork

In [5]:
PICKLE_PATH = "../data/pickled-pork.pickle"
FSIS_DATA = "../data/fsis-processors-with-location.csv"

In [6]:
df = pd.read_csv(FSIS_DATA)
df.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
0,0,M267,4802,JBS,JBS Tolleson Inc.,AZ,Large,Beef,5.0,4.0,"651 S. 91st Ave, Tolleson, AZ 85353",33.441660,-112.252559
1,1,M354,5117,Cargill,Cargill Meat Solutions,CA,Large,Beef,5.0,4.0,"3115 S. Fig Ave., Fresno, CA 93706",36.689651,-119.801950
2,2,M6063A,5289,Central Valley Meat Company,"Central Valley Meat Co., Inc.",CA,Large,Beef,5.0,4.0,"10431 8 3/4 Ave., Hanford, CA 93230",36.321273,-119.612222
3,3,M783 + P783 + V783,5144,Central Valley Meat Company,Harris Ranch Beef Company,CA,Large,Beef,5.0,4.0,"16277 S McCall Ave., Selma, CA 93662",36.499212,-119.614553
4,4,M21488 + V21488,8349,Independent,OWB Packers LLC,CA,Large,Beef,4.0,4.0,"57 East Shank Road, Brawley, CA 92227",33.000969,-115.521786


In [7]:
df["Animals Processed"].value_counts()

Beef,Pork,Sheep/Lamb                   420
Chicken                                184
Beef                                   141
Pork                                   107
Beef,Pork                               92
Beef,Sheep/Lamb                         66
Turkey                                  39
Chicken,Turkey                          29
Turkey,Chicken                          21
Sheep/Lamb                              17
Beef,Pork,Chicken,Turkey,Sheep/Lamb     11
Beef,Pork,Sheep/Lamb,Chicken,Turkey      5
Sheep/Lamb,Pork                          5
Beef,Pork,Chicken,Turkey                 3
Pork,Sheep/Lamb                          3
Beef,Pork,Sheep/Lamb,Chicken             2
Beef,Chicken,Turkey                      1
Pork,Turkey                              1
Sheep/Lamb,Pork,Chicken,Turkey           1
Beef,Sheep/Lamb,Chicken,Turkey           1
Name: Animals Processed, dtype: int64

In [8]:
df_pork = df[df['Animals Processed'].str.contains('Pork')]
len(df_pork)

650

In [9]:
df_large_pork = df_pork.loc[df_pork.Size == "Large"]
df_large_pork.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
40,40,M360 + P360,5118,Smithfield,Smithfield Packaged Meats Corp.,CA,Large,Pork,5.0,4.0,"3049 E. Vernon Avenue, Vernon, CA 90058",34.005840,-118.218488
41,41,M85O + P17775 + V85O,3228,JBS,Swift Pork Company,IA,Large,Pork,5.0,4.0,"600 South Iowa Avenue, Ottumwa, IA 52501",41.002138,-92.394346
42,42,M3S + V3S,3239,JBS,Swift Pork Company,IA,Large,Pork,5.0,4.0,"402 North Tenth Avenue, Marshalltown, IA 50158",42.054400,-92.897911
43,43,M51340 + V51340,6163890,Prestage,Prestage Foods of Iowa,IA,Large,Pork,5.0,4.0,"3183 Hwy 17, Eagle Grove, IA 50533",42.664140,-93.904447
44,44,M46071 + V46071,6163274,Seaboard,SEABOARD TRIUMPH FOODS,IA,Large,Pork,5.0,4.0,"5555 SEABOARD TRIUMPH PARKWAY, SIOUX CITY, IA ...",42.421072,-96.388160


In [10]:
len(df_large_pork)

31

### Save Isochrones to Dataframe

In [11]:
MAPBOX_TOKEN = "pk.eyJ1IjoidG9kZG5pZWYiLCJhIjoiY2xncGpzbmhhMTBwdzNnbXJjNWlzaTY2aCJ9.UhUELBA2iNIDsTN9YESsIw"
ENDPOINT = "https://api.mapbox.com/isochrone/v1/mapbox/driving/"
DRIVING_DISTANCE = str(int(60 * 1609.34))

In [12]:
isochrones = []
for index, row in df_large_pork.iterrows():
    lat = str(row['latitude'])
    lng = str(row['longitude'])

    # add driving radius isochrone to map
    url = ENDPOINT + lng + "," + lat + "?" + "contours_meters=" + DRIVING_DISTANCE + "&access_token=" + MAPBOX_TOKEN
    response = requests.get(url)
    isochrone = Polygon(response.json()['features'][0]['geometry']['coordinates'])
    isochrones.append(isochrone)

In [13]:
df_large_pork = df_large_pork.copy()
df_large_pork["Isochrone"] = isochrones

In [14]:
df_large_pork.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude,Isochrone
40,40,M360 + P360,5118,Smithfield,Smithfield Packaged Meats Corp.,CA,Large,Pork,5.0,4.0,"3049 E. Vernon Avenue, Vernon, CA 90058",34.005840,-118.218488,"POLYGON ((-118.761488 34.669708, -118.764488 3..."
41,41,M85O + P17775 + V85O,3228,JBS,Swift Pork Company,IA,Large,Pork,5.0,4.0,"600 South Iowa Avenue, Ottumwa, IA 52501",41.002138,-92.394346,"POLYGON ((-92.358346 41.754501, -92.360144 41...."
42,42,M3S + V3S,3239,JBS,Swift Pork Company,IA,Large,Pork,5.0,4.0,"402 North Tenth Avenue, Marshalltown, IA 50158",42.054400,-92.897911,"POLYGON ((-92.906911 42.852523, -92.908516 42...."
43,43,M51340 + V51340,6163890,Prestage,Prestage Foods of Iowa,IA,Large,Pork,5.0,4.0,"3183 Hwy 17, Eagle Grove, IA 50533",42.664140,-93.904447,"POLYGON ((-93.988447 43.474354, -93.991447 43...."
44,44,M46071 + V46071,6163274,Seaboard,SEABOARD TRIUMPH FOODS,IA,Large,Pork,5.0,4.0,"5555 SEABOARD TRIUMPH PARKWAY, SIOUX CITY, IA ...",42.421072,-96.388160,"POLYGON ((-96.17516 43.201593, -96.17816 43.20..."


In [15]:
df_large_pork.to_pickle(PICKLE_PATH)